# Was ist neu in Python 3.9?

Mit Python 3.9 wird erstmals auch ein neuer Release-Zyklus verwendet: zukünftig sollen jährlich neue Releases jährlich erscheinen (s.a. [PEP 602](https://www.python.org/dev/peps/pep-0602/)). Die Entwickler\*innen erhoffen sich hiervon schnellere Rückmeldungen bei neuen Features.

Mit dem ersten veröffentlichten Release-Kandidaten soll Python auch eine stabile Binärschnittstelle (engl. *application binary interface,* ABI) erhalten: es soll nun keine ABI-Änderungen mehr in der 3.9-Reihe geben und Erweiterungsmodule müssen nicht mehr für jede neue Version neu kompiliert werden.

Einen vollständigen Überblick erhaltet ihr in [What’s New In Python 3.9](https://docs.python.org/3.9/whatsnew/3.9.html).

Im Folgenden erhaltet ihr ein kurzer Überblick über einige der neuen Features.

## Installation

### Überprüfen

In [1]:
!python3 -V

Python 3.9.0rc1


## [PEP 584](https://www.python.org/dev/peps/pep-0584): Dictionary Merge- und Update-Operatoren

Für die built-in [dict](https://docs.python.org/3.9/library/stdtypes.html#dict)-Klasse gibt es nun ähnliche Operatoren wie zum Verketten von Listen: Merge (`|`) und Update (`|=`). Damit werden verschiedene Nachteile der bisherigen Methoden `dict.update`, `{**d1, **d2}` und `collections.ChainMap` beseitigt. 

### Beispiel [ipykernel/ipykernel/kernelapp.py](https://github.com/ipython/ipykernel/blob/master/ipykernel/kernelapp.py)

In [ ]:
kernel_aliases = dict(base_aliases)
kernel_aliases.update({
    'ip' : 'IPKernelApp.ip',
    'hb' : 'IPKernelApp.hb_port',
    'shell' : 'IPKernelApp.shell_port',
    'iopub' : 'IPKernelApp.iopub_port',
    'stdin' : 'IPKernelApp.stdin_port',
    'control' : 'IPKernelApp.control_port',
    'f' : 'IPKernelApp.connection_file',
    'transport': 'IPKernelApp.transport',
})

kernel_flags = dict(base_flags)
kernel_flags.update({
    'no-stdout' : (
            {'IPKernelApp' : {'no_stdout' : True}},
            "redirect stdout to the null device"),
    'no-stderr' : (
            {'IPKernelApp' : {'no_stderr' : True}},
            "redirect stderr to the null device"),
    'pylab' : (
        {'IPKernelApp' : {'pylab' : 'auto'}},
        """Pre-load matplotlib and numpy for interactive use with
        the default matplotlib backend."""),
    'trio-loop' : (
        {'InteractiveShell' : {'trio_loop' : False}},
        'Enable Trio as main event loop.'
    ),
})

kann vereinfacht werden mit:

In [ ]:
kernel_aliases = base_aliases | {
    'ip': 'KernelApp.ip',
    'hb': 'KernelApp.hb_port',
    'shell': 'KernelApp.shell_port',
    'iopub': 'KernelApp.iopub_port',
    'stdin': 'KernelApp.stdin_port',
    'parent': 'KernelApp.parent',
}}
if sys.platform.startswith ('win'):
    kernel_aliases ['interrupt'] = 'KernelApp.interrupt'

kernel_flags = base_flags | {
    'no-stdout': (
            {'KernelApp': {'no_stdout': True}},
            "stdout auf das Nullgerät umleiten"),
    'no-stderr': (
            {'KernelApp': {'no_stderr': True}},
            "stderr auf das Nullgerät umleiten"),
}}

### Beispiel [matplotlib/legend.py](https://github.com/matplotlib/matplotlib/blob/master/lib/matplotlib/legend.py)

In [ ]:
hm = default_handler_map.copy()
hm.update(self._custom_handler_map)
return hm

kann vereinfacht werden mit:

In [ ]:
return default_handler_map | self._handler_map

## [PEP 616](https://www.python.org/dev/peps/pep-0616): `removeprefix()` und `removesuffix()` für String-Methoden

Mit [`str.removeprefix(prefix)`](https://docs.python.org/3.9/library/stdtypes.html#str.removeprefix) und [`str.removesuffix(suffix)`](https://docs.python.org/3.9/library/stdtypes.html#str.removesuffix) lassen sich nun einfach Präfixe und Suffixe entfernen. Auch für `bytes`, `bytearray`-Objekte  und `collections.UserString`-Zeichenketten wurden ähnliche Methoden hinzugefügt. Insgesamt soll dies zu weniger zerbrechlichem, performanterem und besser lesbarem Code führen.

### Beispiel [find_recursionlimit.py](https://github.com/python/cpython/blob/master/Tools/scripts/find_recursionlimit.py)

In [ ]:
if test_func_name.startswith("test_"):
    print(test_func_name[5:])
else:
    print(test_func_name)

kann vereinfacht werden mit:

In [ ]:
print (test_func_name.removeprefix ("test_"))

### Beispiel [deccheck.py](https://github.com/python/cpython/blob/master/Modules/_decimal/tests/deccheck.py)

In [ ]:
if funcname.startswith("context."):
    self.funcname = funcname.replace("context.", "")
    self.contextfunc = True
else:
    self.funcname = funcname

kann vereinfacht werden mit:

In [ ]:
self.contextfunc = funcname.startswith ("context.")
self.funcname = funcname.removeprefix ("context.")

## [PEP 585](https://www.python.org/dev/peps/pep-0585): Zusätzliche genereiche Typen

In *Type Annotations* können nun Typen wie z.B. `list` oder `dict` als generische Typen direkt verwendet werden. sie müssen nicht mehr extra aus `typing` importiert werden und  das Importieren von `typing` wird damit auch *deprecated*.

### Beispiel

In [ ]:
def greet_all(names: list[str]) -> None:
    for name in names:
        print("Hello", name)

## [PEP 617](https://www.python.org/dev/peps/pep-0617): Neuer PEG-Parser

Python 3.9 verwendet nun einen [PEG](https://en.wikipedia.org/wiki/Parsing_expression_grammar) (Parsing Expression Grammar)-Parser anstelle des bisherigen [LL](https://de.wikipedia.org/wiki/LL-Parser)-Parser. Dies hat u.a. folgende Virteile:
* das Parsen abstrakter Syntaxbäume (AST) vereinfacht sich erheblich
* [Left recursion](https://en.wikipedia.org/wiki/Left_recursion) wird möglich
* Das Erstellen konkreter Syntaxbäume (CST) wird möglich

Der neue Parser ist damit flexibler und soll vor allem beim Entwerfen neuer Sprachfunktionen genutzt werden. Das [ast](https://docs.python.org/3.9/library/ast.html#module-ast)-Modul verwendet den neuen Parser schon jetzt, ohne dass sich an der Ausgabe etwas geändert hätte.

In Python 3.10 wird der alte Parser und alle davon abhängigen Funktionen – hauptsächlich das veraltete [parser](https://docs.python.org/3.9/library/parser.html#module-parser)-Modul – gelöscht. Nur in Python 3.9 könn Ihr mit der Kommandozeile `-X oldparser` oder mit der Umgebungsvariable `PYTHONOLDPARSER=1` zum LL-Parser zurückkehren.